In [3]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
model = Sequential()

def conv2d_bn(x, filters, num_row, num_col, padding="same", strides=(1, 1)):

  # adding conv-2d layer 
  # x = keras.layers.Conv2D(filters, (num_row, num_col),strides=strides,padding=padding)(x)
  model.add(Conv2D(filters, (num_row, num_col),strides=strides,padding=padding))
    
  # adding batch normalization 
  x = keras.layers.BatchNormalization(scale=False)(x)
    
  # adding activation function 
  x = keras.layers.Activation("relu")(x)

  # return x

In [3]:
def inc_block_a(x):    
  
  branch1x1 = conv2d_bn(x, 64, 1, 1)  

  branch5x5 = conv2d_bn(x, 48, 1, 1)  
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

  branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = keras.layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],axis = 3)

  return x

In [4]:
def reduction_block_a(x):  

  branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding="valid")

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding="valid")

  branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
  x = keras.layers.concatenate([branch3x3, branch3x3dbl, branch_pool],axis = 3)

  return x

In [5]:
def inc_block_b(x):

    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1),padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = keras.layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],axis = 3)

    return x

In [6]:
def reduction_block_b(x): 

    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,strides=(2, 2), padding="valid")

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn( branch7x7x3, 192, 3, 3, strides=(2, 2), padding="valid")

    branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = keras.layers.concatenate([branch3x3, branch7x7x3, branch_pool],axis =3)

    return x

In [7]:
def inc_block_c(x):        

        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = keras.layers.concatenate([branch3x3_1, branch3x3_2],axis = 3)

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = keras.layers.concatenate([branch3x3dbl_1, branch3x3dbl_2],axis = 3)

        branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = keras.layers.concatenate( [branch1x1, branch3x3, branch3x3dbl, branch_pool], axis = 3)

        return x

In [8]:

conv2d_bn(keras.Input(shape=(320,320,9)), 32, 3, 3, strides=(2, 2), padding="valid") 
# x = conv2d_bn(x, 32, 3, 3, padding="valid")  
# x = conv2d_bn(x, 64, 3, 3) 
# x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)   
# x = conv2d_bn(x, 80, 1, 1, padding="valid") 
# x = conv2d_bn(x, 192, 3, 3, padding="valid")  
# x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)  

# x=inc_block_a(x) 
# x=inc_block_a(x) 
# x=inc_block_a(x) 

# x=reduction_block_a(x) 

#x=inc_block_b(x) 
#x=inc_block_b(x) 
# x=inc_block_b(x) 
# x=inc_block_b(x) 
# x =reduction_block_b(x) 

# x=inc_block_c(x) 
# x=inc_block_c(x) 

# model.add(Conv2D())
# x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x) 
# x= keras.layers.Flatten()(x)

In [4]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5.csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)


1000


In [5]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
# print(trainAttrX)
# print(testAttrX)
# print(trainy)
# print(testy)

In [6]:
print(testy)
newtesty = []
for value in testy:
    value = float(value)
    newtesty.append(value)
print(newtesty)
newtrainy = []
for value in trainy:
    value = float(value)
    newtrainy.append(value)

print(newtrainy)

529    1
521    1
19     0
896    1
368    0
      ..
986    1
702    1
345    0
996    1
833    1
Name: Gas, Length: 200, dtype: int64
[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0

In [23]:
def create_inception():
    model = Sequential()
    model.add(Conv2D(32,kernel_size = (3,3),input_shape = (320,320,9),strides = (2,2),padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding='same'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(Conv2D(80,kernel_size=(1,1),strides=(1,1),padding='valid'))
    model.add(Conv2D(192,kernel_size=(3,3),strides=(1,1),padding='valid'))
    model.add(MaxPooling2D(pool_size=(3,3),strides = (2,2)))
    # print(model.summary())
    
    #block b
    branch1x1=model
    branch1x1.add(Conv2D(64,kernel_size=(1,1),strides=(1,1),padding='same'))
    branch1x1.add(BatchNormalization())
    #print(branch1x1.summary())
    #print("branch1x1 layer output shape")
    #for layer in branch1x1.layers:
    #   print(layer.output_shape)

    branch7x7 = model
    branch7x7.add(Conv2D(128,kernel_size=(1,1),strides=(1,1),padding='same'))
    branch7x7.add(BatchNormalization())
    branch7x7.add(Conv2D(128,kernel_size=(1,7),strides=(1,1),padding='same'))
    branch7x7.add(BatchNormalization())
    branch7x7.add(Conv2D(192,kernel_size=(7,1),strides=(1,1),padding='same'))
    branch7x7.add(BatchNormalization())
    #print(branch7x7.summary())
    #print("branch7x7 layer output shape")
    #for layer in branch7x7.layers:
    #  print(len(layer.output_shape))
    #   print(layer.output_shape[0])

    branch7x7dbl=model
    branch7x7dbl.add(Conv2D(128,kernel_size=(1,1),strides=(1,1),padding='same'))
    branch7x7dbl.add(BatchNormalization())
    branch7x7dbl.add(Conv2D(128,kernel_size=(7,1),strides=(1,1),padding='same'))
    branch7x7dbl.add(BatchNormalization())
    branch7x7dbl.add(Conv2D(128,kernel_size=(1,7 ),strides=(1,1),padding='same'))
    branch7x7dbl.add(BatchNormalization())
    branch7x7dbl.add(Conv2D(192,kernel_size=(7,1),strides=(1,1),padding='same'))
    branch7x7dbl.add(BatchNormalization())   
    #print(branch7x7dbl.summary())
    #print("branch7x7db1 layer output shape")
    #for layer in branch7x7dbl.layers:
    #   print(len(layer.output_shape))

    branch_pool = model
    branch_pool.add(AveragePooling2D(pool_size=(3,3),strides=(1,1),padding='same'))
    branch_pool.add(Conv2D(192,kernel_size=(1,1),strides=(1,1),padding='same'))
    #print(branch_pool.summary())
    #print("branch_pool layer output shape")
    #for layer in branch_pool.layers:
    #   print(layer.output_shape)

    cancatenate_input = keras.Input(shape=(320,320,9))
    #cancatenate_output = keras.layers.Concatenate()([branch1x1(cancatenate_input),branch7x7(cancatenate_input)])
    cancatenate_output = keras.layers.Concatenate()([branch1x1(cancatenate_input)])
    cancatenate_model = Model(cancatenate_input, cancatenate_output)
    cancatenate_model = Flatten()(cancatenate_model)
    return model



In [34]:
model = Sequential()
model.add(Conv2D(32,kernel_size = (3,3),input_shape = (320,320,9),strides = (2,2),padding = 'valid'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding='same'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(80,kernel_size=(1,1),strides=(1,1),padding='valid'))
model.add(Conv2D(192,kernel_size=(3,3),strides=(1,1),padding='valid'))
model.add(MaxPooling2D(pool_size=(3,3),strides = (2,2)))
# print(model.summary())

#block b
branch1x1=model
branch1x1.add(Conv2D(64,kernel_size=(1,1),strides=(1,1),padding='same'))
branch1x1.add(BatchNormalization())
#print(branch1x1.summary())
#print("branch1x1 layer output shape")
#for layer in branch1x1.layers:
#   print(layer.output_shape)

branch7x7 = model
branch7x7.add(Conv2D(128,kernel_size=(1,1),strides=(1,1),padding='same'))
branch7x7.add(BatchNormalization())
branch7x7.add(Conv2D(128,kernel_size=(1,7),strides=(1,1),padding='same'))
branch7x7.add(BatchNormalization())
branch7x7.add(Conv2D(192,kernel_size=(7,1),strides=(1,1),padding='same'))
branch7x7.add(BatchNormalization())
#print(branch7x7.summary())
#print("branch7x7 layer output shape")
#for layer in branch7x7.layers:
#  print(len(layer.output_shape))
#   print(layer.output_shape[0])

branch7x7dbl=model
branch7x7dbl.add(Conv2D(128,kernel_size=(1,1),strides=(1,1),padding='same'))
branch7x7dbl.add(BatchNormalization())
branch7x7dbl.add(Conv2D(128,kernel_size=(7,1),strides=(1,1),padding='same'))
branch7x7dbl.add(BatchNormalization())
branch7x7dbl.add(Conv2D(128,kernel_size=(1,7 ),strides=(1,1),padding='same'))
branch7x7dbl.add(BatchNormalization())
branch7x7dbl.add(Conv2D(192,kernel_size=(7,1),strides=(1,1),padding='same'))
branch7x7dbl.add(BatchNormalization())   
#print(branch7x7dbl.summary())
#print("branch7x7db1 layer output shape")
#for layer in branch7x7dbl.layers:
#   print(len(layer.output_shape))

branch_pool = model
branch_pool.add(AveragePooling2D(pool_size=(3,3),strides=(1,1),padding='same'))
branch_pool.add(Conv2D(192,kernel_size=(1,1),strides=(1,1),padding='same'))
#print(branch_pool.summary())
#print("branch_pool layer output shape")
#for layer in branch_pool.layers:
#   print(layer.output_shape)

cancatenate_input = keras.Input(shape=(320,320,9))
#cancatenate_output = keras.layers.Concatenate()([branch1x1(cancatenate_input),branch7x7(cancatenate_input)])
cancatenate_output = keras.layers.Concatenate()([branch1x1(cancatenate_input)])
cancatenate_model = Model(cancatenate_input, cancatenate_output)

In [35]:
model.add(Flatten())
X=model.predict(trainImagesX)

25/25 [==============================] - 8s 307ms/step


In [36]:
X = np.array(X)
trainAttrX = np.array(trainAttrX)
trainAttrX.resize(800,9216)
X.resize(800,554496)
newtrainy=np.array(newtrainy)
newtrainy = newtrainy.reshape(-1,1)
#print(X)
#print(X.shape)
#print(trainAttrX)
print(trainAttrX.shape)
traindata = concatenate([X,trainAttrX])
traindata = np.array(traindata)
traindata.reshape(-1,1)
print(traindata)
print(traindata.shape)



(800, 9216)
[[-2.64727278e-04  1.26642437e-04  5.50581724e-04 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.72156127e-05 -3.76027456e-05  3.78266559e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.34997307e-05 -2.63210368e-05  1.20907374e-04 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
(800, 563712)


In [37]:
mod = CascadeForestClassifier(n_estimators=50,random_state=1)
mod.fit(traindata,trainy)

In [ ]:
cnnpred = cnn.predict(trainImagesX)
model.predict()